# Cat Dog Chicken Baby


This code is based on code from a fast.ai MOOC that will be publicly available in Jan 2019.

Any code here is free to use however please ensure you credit Fast.Ai.
All corrections, modifications and collaborations are welcome.

This notebook is built on the first Fast.Ai lesson and is hopefully able to tell the difference between Cats, Dogs, Chickens and Babies.

Lets begin,

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Quick check to make sure we have a functioning CUDA to train with.

CPU training options are provided by the fastai library but are painfully slow.

In [1]:
torch.cuda.is_available()

NameError: name 'torch' is not defined

This notebook is based off the [fastai V1 library](http://www.fast.ai/2018/10/02/fastai-ai/) which sits on top of [Pytorch 1.0](https://hackernoon.com/pytorch-1-0-468332ba5163). The fastai library provides many useful functions that enable us to quickly and easily build neural networks and train our models.

In [ ]:
from fastai import *
from fastai.vision import *

Refer to the Image Downloader notebook to obtain images.

This notebook assumes that your training/validation images are held in the data folder which should be in the same directory as this notebook. 

The first step it to load up our images and obtain a list of classes from the filenames. This is done using regex and `ImageDataBunch.from_name_re`

Lets load in our images and check that they can be loaded.

In [ ]:
path = Path('data/')
fnames = get_image_files(path_img)
fnames[:5]

In [ ]:
np.random.seed(2)
pat = r'/([^/]+)_\d+.jpg$'

If you get CUDA out of memory errors try reducing the batchsize (bs) to 32 then 16, ensure you `Kernal -> Restart` before retrying.

Ok, lets load some images!

In [ ]:
data = ImageDataBunch.from_name_re(path, fnames, pat, ds_tfms=get_transforms(), size=224, bs=64)
data.normalize(imagenet_stats)
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

# Training

We will begin by creating a leaner from our data batches and train for 4 epochs

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

# Initial results

A quick confusion matrix to see how we did (at the time of writing I'm not in a position actually run any training so i'm just assuming/hoping we have decent accuracy at this point).

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

# Unfreezing, fine-tuning, and learning rates

Now we go ahead and unfreeze the last few layers of our model to better fit it to our training data.

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-4))

# Putting the notebook into production

In [ ]:
data.classes

In [ ]:
# fastai.defaults.device = torch.device('cpu')

In [ ]:
img = open_image(path/'black'/'00000021.jpg')
img

In [ ]:
classes = ['black', 'grizzly', 'teddys']
data2 = ImageDataBunch.single_from_classes(path, classes, tfms=get_transforms(), size=224).normalize(imagenet_stats)
learn = create_cnn(data2, models.resnet34)
learn.load('stage-2')

In [ ]:
pred_class,pred_idx,outputs = learn.predict(img)
pred_class

Stay tuned for better/more interesting models including web based deployment and layer visualisations.